In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import current_timestamp, lit

files = dbutils.fs.ls('/Volumes/workspace/vr_pronssi/vr_pronssi_volume')
latest_file = sorted(files, key=lambda x: x.modificationTime, reverse=True)[0]

log_df = spark.table('vr_pronssi.ingestion_log')
row_count = log_df.count()

if row_count == 0:
    latest_file_path = None
else:
    latest_file_path = spark.sql('''
        SELECT file_path
        FROM workspace.vr_pronssi.ingestion_log
        ORDER BY processed_at DESC
        LIMIT 1                           
    ''').collect()[0]['file_path']

if latest_file_path == latest_file.path:
    dbutils.notebook.exit('EI SUORITETA NOTEBOOKIA. DELTATAULUSSA ON JO TARVITTAVA DATA')



In [0]:
df = spark.read.parquet(latest_file.path)
if df.isEmpty():
  raise Exception("VALIDOINTI EPÄONNISTUI: TUOREIMMASSA TIEDOSTOSSA EI OLLUT DATAA")

display(df.limit(5))

departureDate,trainNumber,trainCategory,trainType,commuterLineID,stationShortCode,type,scheduledTime,actualTime,differenceInMinutes,operatorUICCode,operatorShortCode,cancelled
2026-01-26,1,Long-distance,IC,,HKI,DEPARTURE,2026-01-26T04:54:00.000Z,2026-01-26T04:54:26.000Z,0.0,10,vr,false
2026-01-26,1,Long-distance,IC,,PSL,ARRIVAL,2026-01-26T04:59:00.000Z,2026-01-26T04:58:51.000Z,0.0,10,vr,false
2026-01-26,1,Long-distance,IC,,PSL,DEPARTURE,2026-01-26T05:00:00.000Z,2026-01-26T05:00:47.000Z,1.0,10,vr,false
2026-01-26,1,Long-distance,IC,,LOP,ARRIVAL,2026-01-26T05:00:53.000Z,null,1.0,10,vr,false
2026-01-26,1,Long-distance,IC,,LOP,DEPARTURE,2026-01-26T05:00:53.000Z,null,1.0,10,vr,false


In [0]:
df.write.format('delta').mode('append').saveAsTable('vr_pronssi.vr_raw')

In [0]:
log_data = [Row(file_path = latest_file.path)]
log_df = spark.createDataFrame(log_data)
row_count = df.count()

log_df.withColumn('processed_at', current_timestamp()) \
    .withColumn('new_rows', lit(row_count)) \
    .write \
    .format('delta') \
    .mode('append') \
    .saveAsTable('vr_pronssi.ingestion_log')

dbutils.notebook.exit('PRONSSITASON DATA JA LOKI KIRJOITETTU ONNISTUNEESTI')